In [83]:
import tensorflow as tf
import pandas as pd

In [84]:
train_dataset_fp = pd.read_csv('../iris_training.csv')
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_dataset_fp.iloc[:, :4].values, train_dataset_fp.iloc[:, 4].values))
train_dataset = train_dataset.batch(32)

In [85]:
class MyDense(tf.keras.layers.Layer):
    """自定义全连接层"""

    def __init__(self, units=32, **kwargs):
        self.units = units
        super(MyDense, self).__init__(**kwargs)

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='w')
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='b')

    def get_config(self):
        """tf.keras.models.load_model方法加载.h5格式时需要自定义get_config方法"""
        config = super(MyDense, self).get_config()
        config.update({'units': self.units})  # 添加初始化属性;参考tf.keras.layers.Dense的实现
        return config  # 返回一个JSON可序列化字典

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [86]:
# Model
def get_model():
    inputs = tf.keras.Input(shape=(4,), name='dcdmm')
    x = MyDense(units=10)(inputs)
    x = tf.nn.relu(x)
    x = MyDense(units=10)(x)
    x = tf.nn.relu(x)
    outputs = MyDense(units=3)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

In [87]:
model = get_model()
model.fit(train_dataset, epochs=100)

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dcdmm (InputLayer)          [(None, 4)]               0         
                                                                 
 my_dense_30 (MyDense)       (None, 10)                50        
                                                                 
 tf.nn.relu_20 (TFOpLambda)  (None, 10)                0         
                                                                 
 my_dense_31 (MyDense)       (None, 10)                110       
                                                                 
 tf.nn.relu_21 (TFOpLambda)  (None, 10)                0         
                                                                 
 my_dense_32 (MyDense)       (None, 3)                 33        
                                                                 
Total params: 193
Trainable params: 193
Non-trainable para

In [88]:
predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5, ],
    [5.9, 3.0, 4.2, 1.5, ],
    [6.9, 3.1, 5.4, 2.1]
])

model.predict(predict_dataset)

array([[ 3.6050358,  0.8329937, -4.7134743],
       [-2.2593663,  1.5720733,  0.6705388],
       [-4.7718887,  1.9856163,  2.856376 ]], dtype=float32)

### 方法一:save_weights

In [89]:
model.save_weights('save_weights.h5')

In [90]:
model_k_h5 = get_model()  # 重新创建网络并build
model_k_h5.build(input_shape=(None, 4))

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dcdmm (InputLayer)          [(None, 4)]               0         
                                                                 
 my_dense_33 (MyDense)       (None, 10)                50        
                                                                 
 tf.nn.relu_22 (TFOpLambda)  (None, 10)                0         
                                                                 
 my_dense_34 (MyDense)       (None, 10)                110       
                                                                 
 tf.nn.relu_23 (TFOpLambda)  (None, 10)                0         
                                                                 
 my_dense_35 (MyDense)       (None, 3)                 33        
                                                                 
Total params: 193
Trainable params: 193
Non-trainable para

In [91]:
model_k_h5.load_weights("save_weights.h5")

In [92]:
model_k_h5.predict(predict_dataset)

array([[ 3.6050358,  0.8329937, -4.7134743],
       [-2.2593663,  1.5720733,  0.6705388],
       [-4.7718887,  1.9856163,  2.856376 ]], dtype=float32)

### 方法二:save

In [93]:
# ★★★★★推荐保存为Tensorflow SavedModel格式
model.save('keras_model_tf_version', save_format='tf')

INFO:tensorflow:Assets written to: keras_model_tf_version\assets


In [94]:
new_model_tf = tf.keras.models.load_model('keras_model_tf_version/')
new_model_tf.predict(predict_dataset)

array([[ 3.6050358,  0.8329937, -4.7134743],
       [-2.2593663,  1.5720733,  0.6705388],
       [-4.7718887,  1.9856163,  2.856376 ]], dtype=float32)

In [95]:
model.save('keras_model_tf_version.h5')

In [96]:
# At loading time, register the custom objects with a `custom_object_scope`:
_custom_objects = {
    "MyDense": MyDense,

}

# 自定义层需要自定义层的get_config方法
new_model_h5 = tf.keras.models.load_model('keras_model_tf_version.h5', custom_objects=_custom_objects)
new_model_h5.predict(predict_dataset)

array([[ 3.6050358,  0.8329937, -4.7134743],
       [-2.2593663,  1.5720733,  0.6705388],
       [-4.7718887,  1.9856163,  2.856376 ]], dtype=float32)

In [97]:
# 一般用于模型部署
tf.saved_model.save(model, 'tf_saved_model_version')

INFO:tensorflow:Assets written to: tf_saved_model_version\assets


In [98]:
!saved_model_cli show --dir tf_saved_model_version --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dcdmm'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 4)
        name: serving_default_dcdmm:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['my_dense_32'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 3)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inputs: TensorSpec(shape=(N

In [99]:
restored_saved_model = tf.saved_model.load('tf_saved_model_version')
f = restored_saved_model.signatures["serving_default"]
f(dcdmm=tf.constant(predict_dataset))



{'my_dense_32': <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[ 3.6050358,  0.8329937, -4.7134743],
        [-2.2593663,  1.5720733,  0.6705388],
        [-4.7718887,  1.9856163,  2.856376 ]], dtype=float32)>}